In [1]:
import requests
import json
import pandas as pd
import matplotlib.pyplot as plt

In [2]:
def get_traffic_deaths_data():
    
    # build url
    url = f'https://ec.europa.eu/eurostat/api/dissemination/statistics/1.0/data/sdg_11_40?format=JSON&unit=RT&tra_infr=TOTAL&tra_infr=MWAY&tra_infr=RD_URB&tra_infr=RD_RUR&tra_infr=UNK&lang=en'
    
    url_text = requests.get(url)

    traffic_dict = json.loads(url_text.text)

    return traffic_dict

In [3]:
data = get_traffic_deaths_data()
data

{'version': '2.0',
 'class': 'dataset',
 'label': 'Road traffic deaths, by type of roads',
 'source': 'ESTAT',
 'updated': '2025-05-12T23:00:00+0200',
 'value': {'1344': 1.6,
  '1345': 1.9,
  '1346': 1.6,
  '1347': 1.3,
  '1348': 1.4,
  '1349': 1.1,
  '1350': 0.9,
  '1351': 0.9,
  '1352': 0.9,
  '1353': 0.7,
  '1354': 0.7,
  '1355': 0.5,
  '1356': 0.6,
  '1357': 0.4,
  '1358': 0.4,
  '1359': 0.5,
  '1360': 0.4,
  '1361': 0.5,
  '1362': 0.3,
  '1363': 0.3,
  '1364': 0.3,
  '1365': 0.4,
  '1366': 0.3,
  '1367': 0.4,
  '888': 2.3,
  '889': 1.9,
  '890': 1.7,
  '891': 1.3,
  '892': 1.2,
  '893': 1.5,
  '894': 1.6,
  '895': 1.4,
  '896': 1.3,
  '897': 1.4,
  '898': 1.0,
  '899': 1.2,
  '900': 1.0,
  '901': 1.1,
  '902': 1.0,
  '903': 1.0,
  '904': 0.9,
  '905': 0.9,
  '906': 0.9,
  '907': 1.0,
  '908': 0.7,
  '909': 0.7,
  '910': 0.7,
  '911': 0.7,
  '920': 0.5,
  '921': 0.5,
  '922': 0.5,
  '923': 0.5,
  '924': 0.3,
  '925': 0.3,
  '926': 0.6,
  '927': 0.9,
  '928': 0.6,
  '929': 0.9,
  '9

In [4]:
print(len(data["value"]))
data["dimension"]

3651


{'freq': {'label': 'Time frequency',
  'category': {'index': {'A': 0}, 'label': {'A': 'Annual'}}},
 'tra_infr': {'label': 'Transport infrastructure',
  'category': {'index': {'TOTAL': 0,
    'MWAY': 1,
    'RD_URB': 2,
    'RD_RUR': 3,
    'UNK': 4},
   'label': {'TOTAL': 'Total',
    'MWAY': 'Motorways',
    'RD_URB': 'Urban roads',
    'RD_RUR': 'Rural roads',
    'UNK': 'Unknown'}}},
 'unit': {'label': 'Unit of measure',
  'category': {'index': {'RT': 0}, 'label': {'RT': 'Rate'}}},
 'geo': {'label': 'Geopolitical entity (reporting)',
  'category': {'index': {'EU27_2020': 0,
    'EA20': 1,
    'BE': 2,
    'BG': 3,
    'CZ': 4,
    'DK': 5,
    'DE': 6,
    'EE': 7,
    'IE': 8,
    'EL': 9,
    'ES': 10,
    'FR': 11,
    'HR': 12,
    'IT': 13,
    'CY': 14,
    'LV': 15,
    'LT': 16,
    'LU': 17,
    'HU': 18,
    'MT': 19,
    'NL': 20,
    'AT': 21,
    'PL': 22,
    'PT': 23,
    'RO': 24,
    'SI': 25,
    'SK': 26,
    'FI': 27,
    'SE': 28,
    'IS': 29,
    'LI': 30,
   

In [6]:
def decode_dataset_index(flat_index):
    sizes = data["size"]
    dimension_ids = data["id"]
    dimension_data = data["dimension"]


    # Convert to coordinates
    coords = [0] * len(sizes)
    remaining = flat_index
    
    for i in range(len(sizes) - 1, -1, -1):
        coords[i] = remaining % sizes[i]
        remaining = remaining // sizes[i]
    
    # Map to codes
    result = {}
    for i, dim_id in enumerate(dimension_ids):
        coord = coords[i]
        categories = dimension_data[dim_id]['category']
        
        # Find code for this coordinate
        code = next(k for k, v in categories['index'].items() if v == coord)
        
        result[dim_id] = {
            'code': code,
            'label': categories.get('label', {}).get(code, code),
            'index': coord
        }
    
    return result

decode_dataset_index(200)

{'freq': {'code': 'A', 'label': 'Annual', 'index': 0},
 'tra_infr': {'code': 'TOTAL', 'label': 'Total', 'index': 0},
 'unit': {'code': 'RT', 'label': 'Rate', 'index': 0},
 'geo': {'code': 'IE', 'label': 'Ireland', 'index': 8},
 'time': {'code': '2008', 'label': '2008', 'index': 8}}

In [12]:
new_road_deaths_dict = {'Transport Infrastructure': [], 'Country': [], 'Year': [], 'Road Deaths (per 100k)': []}

for key in data['value'].keys():
    key_info = decode_dataset_index(int(key))

    new_road_deaths_dict['Transport Infrastructure'].append(key_info['tra_infr']['label'])
    new_road_deaths_dict['Country'].append(key_info['geo']['label'])
    new_road_deaths_dict['Year'].append(key_info['time']['label'])
    new_road_deaths_dict['Road Deaths (per 100k)'].append(data['value'][key])

new_road_deaths_df = pd.DataFrame(new_road_deaths_dict)
new_road_deaths_df

,Transport Infrastructure,Country,Year,Road Deaths (per 100k)
0,Motorways,Austria,2000,1.6
1,Motorways,Austria,2001,1.9
2,Motorways,Austria,2002,1.6
3,Motorways,Austria,2003,1.3
4,Motorways,Austria,2004,1.4
...,...,...,...,...
3646,Unknown,United Kingdom,2014,0.0
3647,Unknown,United Kingdom,2015,0.0
3648,Unknown,United Kingdom,2016,0.0
3649,Unknown,United Kingdom,2017,0.0
